# Topic 9: Multi-Agent Systems

Build sophisticated multi-agent systems where multiple AI agents collaborate to solve complex problems. Learn how to coordinate agents, manage their interactions, and create emergent behaviors.

## Learning Objectives

- Design multi-agent architectures
- Implement agent coordination patterns
- Create specialized agents with distinct roles
- Build collaborative problem-solving systems

In [ ]:
# Setup
import os
import getpass
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

model = ChatAnthropic(model="claude-sonnet-4-20250514")
print("✓ Setup complete!")

## Understanding Multi-Agent Systems

Multi-agent systems involve multiple specialized agents working together:

**Key Patterns:**
- **Sequential**: Agents work one after another in a pipeline
- **Parallel**: Multiple agents work simultaneously
- **Hierarchical**: Supervisor agents coordinate worker agents
- **Collaborative**: Agents negotiate and collaborate dynamically

**Benefits:**
- Specialization: Each agent focuses on its expertise
- Scalability: Add new agents without changing existing ones
- Robustness: System continues if one agent fails
- Quality: Multiple perspectives improve outcomes

## Example 1: Research Team (Sequential Multi-Agent)

Let's build a research team with specialized agents:
- Researcher: Gathers information
- Analyst: Analyzes findings
- Writer: Creates final report

In [ ]:
# Define state
class ResearchTeamState(TypedDict):
    topic: str
    research_findings: str
    analysis: str
    final_report: str

print("✓ ResearchTeamState defined")

In [ ]:
def researcher_agent(state: ResearchTeamState) -> ResearchTeamState:
    """Agent that researches the topic."""
    print("\n🔬 Researcher Agent: Gathering information...")
    
    system_msg = """You are a research specialist. Your role is to:
    - Gather comprehensive information on topics
    - Identify key facts and trends
    - Cite credible sources
    - Organize findings clearly"""
    
    prompt = f"Research this topic thoroughly: {state['topic']}"
    
    response = model.invoke([
        SystemMessage(content=system_msg),
        HumanMessage(content=prompt)
    ])
    
    print("✓ Research complete")
    return {"research_findings": response.content}

def analyst_agent(state: ResearchTeamState) -> ResearchTeamState:
    """Agent that analyzes the research."""
    print("\n📊 Analyst Agent: Analyzing findings...")
    
    system_msg = """You are a data analyst. Your role is to:
    - Analyze research findings critically
    - Identify patterns and insights
    - Draw meaningful conclusions
    - Highlight implications"""
    
    prompt = f"""Analyze these research findings:
    
{state['research_findings']}

Provide deep analysis with insights and implications."""
    
    response = model.invoke([
        SystemMessage(content=system_msg),
        HumanMessage(content=prompt)
    ])
    
    print("✓ Analysis complete")
    return {"analysis": response.content}

def writer_agent(state: ResearchTeamState) -> ResearchTeamState:
    """Agent that writes the final report."""
    print("\n✍️  Writer Agent: Creating final report...")
    
    system_msg = """You are a professional writer. Your role is to:
    - Create clear, engaging reports
    - Synthesize research and analysis
    - Use proper structure and flow
    - Write for the target audience"""
    
    prompt = f"""Create a comprehensive report using:
    
RESEARCH:
{state['research_findings']}

ANALYSIS:
{state['analysis']}

Write a clear, well-structured report (500-600 words)."""
    
    response = model.invoke([
        SystemMessage(content=system_msg),
        HumanMessage(content=prompt)
    ])
    
    print("✓ Report complete")
    return {"final_report": response.content}

print("✓ All agent nodes created")

In [ ]:
# Build the sequential multi-agent graph
research_team_builder = StateGraph(ResearchTeamState)

# Add agent nodes
research_team_builder.add_node("researcher", researcher_agent)
research_team_builder.add_node("analyst", analyst_agent)
research_team_builder.add_node("writer", writer_agent)

# Create sequential flow
research_team_builder.add_edge(START, "researcher")
research_team_builder.add_edge("researcher", "analyst")
research_team_builder.add_edge("analyst", "writer")
research_team_builder.add_edge("writer", END)

# Compile
research_team_graph = research_team_builder.compile()

print("✓ Research team graph compiled!")

In [ ]:
from IPython.display import Image, display

try:
    display(Image(research_team_graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph: START -> researcher -> analyst -> writer -> END")

In [ ]:
# Run the research team
print("Starting research team workflow...")
print("="*60)

result = research_team_graph.invoke({
    "topic": "The Impact of AI Agents on Business Automation",
    "research_findings": "",
    "analysis": "",
    "final_report": ""
})

print("\n" + "="*60)
print("FINAL REPORT")
print("="*60)
print(result['final_report'])

## Example 2: Supervisor Pattern

A supervisor agent coordinates multiple worker agents and decides which agent to call next.

In [ ]:
class SupervisorState(TypedDict):
    messages: Annotated[list, add_messages]
    next_agent: str
    task_complete: bool

print("✓ SupervisorState defined")

In [ ]:
# Specialized worker agents
def code_expert(state: SupervisorState) -> SupervisorState:
    """Expert in writing and reviewing code."""
    print("\n💻 Code Expert: Working...")
    
    system_msg = "You are a senior software engineer. Help with code-related tasks."
    
    response = model.invoke([
        SystemMessage(content=system_msg),
        *state['messages']
    ])
    
    return {"messages": [response]}

def documentation_expert(state: SupervisorState) -> SupervisorState:
    """Expert in writing documentation."""
    print("\n📝 Documentation Expert: Working...")
    
    system_msg = "You are a technical writer. Help with documentation tasks."
    
    response = model.invoke([
        SystemMessage(content=system_msg),
        *state['messages']
    ])
    
    return {"messages": [response]}

def testing_expert(state: SupervisorState) -> SupervisorState:
    """Expert in testing and quality assurance."""
    print("\n🧪 Testing Expert: Working...")
    
    system_msg = "You are a QA engineer. Help with testing and quality tasks."
    
    response = model.invoke([
        SystemMessage(content=system_msg),
        *state['messages']
    ])
    
    return {"messages": [response]}

print("✓ Worker agents created")

In [ ]:
def supervisor(state: SupervisorState) -> SupervisorState:
    """Supervisor that routes to appropriate agent."""
    print("\n👔 Supervisor: Analyzing request...")
    
    system_msg = """You are a project supervisor managing three experts:
    - code_expert: For coding and programming tasks
    - documentation_expert: For documentation and writing
    - testing_expert: For testing and quality assurance
    
    Analyze the request and respond with ONLY the name of the expert who should handle it.
    If the task is complete, respond with 'FINISH'."""
    
    response = model.invoke([
        SystemMessage(content=system_msg),
        *state['messages']
    ])
    
    next_agent = response.content.strip().lower()
    
    if 'finish' in next_agent:
        print("   → Task complete!")
        return {"next_agent": "FINISH", "task_complete": True}
    
    print(f"   → Routing to: {next_agent}")
    return {"next_agent": next_agent}

print("✓ Supervisor created")

In [ ]:
def route_to_agent(state: SupervisorState) -> Literal["code_expert", "documentation_expert", "testing_expert", "supervisor", "end"]:
    """Router based on supervisor's decision."""
    next_agent = state.get('next_agent', '')
    
    if state.get('task_complete', False) or next_agent == 'FINISH':
        return "end"
    elif 'code' in next_agent:
        return "code_expert"
    elif 'documentation' in next_agent or 'doc' in next_agent:
        return "documentation_expert"
    elif 'testing' in next_agent or 'test' in next_agent:
        return "testing_expert"
    else:
        # Default back to supervisor
        return "supervisor"

print("✓ Router created")

In [ ]:
# Build supervisor graph
supervisor_builder = StateGraph(SupervisorState)

# Add all nodes
supervisor_builder.add_node("supervisor", supervisor)
supervisor_builder.add_node("code_expert", code_expert)
supervisor_builder.add_node("documentation_expert", documentation_expert)
supervisor_builder.add_node("testing_expert", testing_expert)

# Start with supervisor
supervisor_builder.add_edge(START, "supervisor")

# Supervisor routes to workers
supervisor_builder.add_conditional_edges(
    "supervisor",
    route_to_agent,
    {
        "code_expert": "code_expert",
        "documentation_expert": "documentation_expert",
        "testing_expert": "testing_expert",
        "end": END
    }
)

# Workers report back to supervisor
supervisor_builder.add_edge("code_expert", "supervisor")
supervisor_builder.add_edge("documentation_expert", "supervisor")
supervisor_builder.add_edge("testing_expert", "supervisor")

# Compile
supervisor_graph = supervisor_builder.compile()

print("✓ Supervisor graph compiled!")

In [ ]:
# Test the supervisor system
print("Testing supervisor system...")
print("="*60)

result = supervisor_graph.invoke({
    "messages": [HumanMessage(content="Write a Python function to calculate fibonacci numbers")],
    "next_agent": "",
    "task_complete": False
})

print("\n" + "="*60)
print("CONVERSATION:")
print("="*60)
for msg in result['messages']:
    role = "User" if isinstance(msg, HumanMessage) else "Agent"
    print(f"\n{role}: {msg.content[:200]}..." if len(msg.content) > 200 else f"\n{role}: {msg.content}")

## Example 3: Debate Between Agents

Create agents that debate different perspectives to reach better conclusions.

In [ ]:
class DebateState(TypedDict):
    topic: str
    pro_argument: str
    con_argument: str
    moderator_summary: str
    rounds: int

def pro_agent(state: DebateState) -> DebateState:
    """Agent arguing FOR the topic."""
    print("\n✅ Pro Agent: Presenting argument...")
    
    context = f"Con argument: {state.get('con_argument', 'None yet')}" if state.get('con_argument') else ""
    
    prompt = f"""Debate topic: {state['topic']}
    
You are arguing FOR this position. Present strong, logical arguments.
{context}

Provide your argument (keep it under 200 words):"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    return {"pro_argument": response.content, "rounds": state.get('rounds', 0) + 1}

def con_agent(state: DebateState) -> DebateState:
    """Agent arguing AGAINST the topic."""
    print("\n❌ Con Agent: Presenting counter-argument...")
    
    prompt = f"""Debate topic: {state['topic']}
    
You are arguing AGAINST this position. Present strong, logical arguments.
Pro argument: {state['pro_argument']}

Provide your counter-argument (keep it under 200 words):"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    return {"con_argument": response.content}

def moderator_agent(state: DebateState) -> DebateState:
    """Moderator that summarizes the debate."""
    print("\n⚖️  Moderator: Summarizing debate...")
    
    prompt = f"""Topic: {state['topic']}
    
PRO ARGUMENT:
{state['pro_argument']}

CON ARGUMENT:
{state['con_argument']}

As a neutral moderator, provide:
1. Summary of both positions
2. Strength of each argument
3. Balanced conclusion"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    return {"moderator_summary": response.content}

print("✓ Debate agents created")

In [ ]:
# Build debate graph
debate_builder = StateGraph(DebateState)

debate_builder.add_node("pro", pro_agent)
debate_builder.add_node("con", con_agent)
debate_builder.add_node("moderator", moderator_agent)

debate_builder.add_edge(START, "pro")
debate_builder.add_edge("pro", "con")
debate_builder.add_edge("con", "moderator")
debate_builder.add_edge("moderator", END)

debate_graph = debate_builder.compile()

print("✓ Debate graph compiled!")

In [ ]:
# Run a debate
print("Starting debate...")
print("="*60)

result = debate_graph.invoke({
    "topic": "AI agents should replace traditional software applications",
    "pro_argument": "",
    "con_argument": "",
    "moderator_summary": "",
    "rounds": 0
})

print("\n" + "="*60)
print("DEBATE SUMMARY")
print("="*60)
print(result['moderator_summary'])

## Key Multi-Agent Patterns

### 1. Sequential Pipeline
Agents work one after another, each building on previous work

### 2. Supervisor Pattern
Central coordinator routes tasks to specialized workers

### 3. Peer Collaboration
Agents work together as equals, debating and refining

### 4. Hierarchical Teams
Multiple levels of agents with different responsibilities

## Exercise: Build a Software Development Team

Create a multi-agent system with:
1. Product Manager: Defines requirements
2. Architect: Designs system architecture
3. Developer: Writes code
4. Reviewer: Reviews and provides feedback
5. QA: Tests the solution

Use the supervisor pattern with a team lead coordinating the agents.

In [ ]:
# Your code here!

class DevTeamState(TypedDict):
    feature_request: str
    requirements: str
    architecture: str
    code: str
    review_feedback: str
    test_results: str
    next_agent: str

# TODO: Create agent nodes for each role
# TODO: Create team lead supervisor
# TODO: Build graph with coordination
# TODO: Test with a feature request

## Key Takeaways

In this notebook, you learned:

1. ✅ Designing multi-agent architectures with specialized roles
2. ✅ Implementing sequential agent pipelines
3. ✅ Building supervisor patterns for agent coordination
4. ✅ Creating collaborative debate systems
5. ✅ Using system messages to define agent personalities
6. ✅ Routing between agents based on task requirements

## Next Steps

Continue to **Topic 10: Advanced Patterns** to learn production-ready patterns and best practices!